In [1]:
from langchain_community.document_loaders import YoutubeLoader

In [2]:
loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=mrKuDK9dGlg&list=PLrLEqwuz-mRIEtuUEN8sse2XyksKNN4Om", add_video_info=False,
    language=["en", "id"],
    translation="en",
)
d = loader.load()
d[0].page_content

"hello everyone welcome to AI anytime  channel so I'm starting a new playlist  called finetuning of large language  models in this playlist I will cover you  know the different aspect of fine-tuning  a large language model how to you know  finetune on your data set you will have  your custom data sets you want to fine  tune llms in this entire playlist we'll  have multiple videos we'll start with  the fundamentals and we starting in this  video as well the fundamentals you know  the different steps of pre-training  fine-tuning and the Noel techniques like  Laura and Kora how to select the right  hyper parameters for fine tuning task  what other tools that you can use to  fine tune like Excel toll for example  that we'll cover in this video which is  a low code fine-tuning tool and you know  how to use Laura and Kora to fine tune  uh with reduced memory uh consumption  now in this playlist we'll have videos  to F tune Lama 2 F tune Mistral fine  tune open Lama and fine tune other open  

In [3]:
d

[Document(page_content="hello everyone welcome to AI anytime  channel so I'm starting a new playlist  called finetuning of large language  models in this playlist I will cover you  know the different aspect of fine-tuning  a large language model how to you know  finetune on your data set you will have  your custom data sets you want to fine  tune llms in this entire playlist we'll  have multiple videos we'll start with  the fundamentals and we starting in this  video as well the fundamentals you know  the different steps of pre-training  fine-tuning and the Noel techniques like  Laura and Kora how to select the right  hyper parameters for fine tuning task  what other tools that you can use to  fine tune like Excel toll for example  that we'll cover in this video which is  a low code fine-tuning tool and you know  how to use Laura and Kora to fine tune  uh with reduced memory uh consumption  now in this playlist we'll have videos  to F tune Lama 2 F tune Mistral fine  tune open Lama and

In [62]:
from youtube_transcript_api import YouTubeTranscriptApi

In [4]:
from pydantic import BaseModel
from dotenv import load_dotenv
import google.generativeai as genai
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.vectorstores import Chroma  # chroma for storing vector store locally
from langchain_community.embeddings import HuggingFaceBgeEmbeddings # for converting text to embedings
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

load_dotenv()
api =os.getenv("GOOGLE_API_KEY") #uncomment this during locally
genai.configure(api_key=api) 

c:\Users\hp\anaconda3\envs\geminienv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
load_dotenv()

True

In [8]:
def get_transcript(url:str):
     """
     Try to get the transcription of the yt video
     input: url
     output: yt_transcription
     """
     try:
        loader = YoutubeLoader.from_youtube_url(
            url, add_video_info=False,
        language=["en", "hi"],
        translation="en",
        )
        doc = loader.load()
        return doc
     except:
        return "not able to get transcript"

In [9]:
def yt_transcript(video_id):
    try:
        transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
        for transcript in transcript_list:
            transcript_text =transcript.translate('en').fetch()
        return transcript_text

    except Exception as e:
        raise e


In [39]:
trans_from_get = get_transcript("https://www.youtube.com/watch?v=QQfZAoNGQmE&t=2178s")
trans_from_get

[Document(page_content="Hi Guys my name is Nitesh and you welcome to mine its name is GRU Gated Recurrent Unit so in this video we will understand in great detail first of all if LSTM was performing so well then there is a need to use GR.  Why and after that we will understand its architecture in great detail. To explain the architecture, I will explain with examples and finally we will see the key differences between LSTM and GR usage. Okay, the video may be a little long because it has been explained in great detail. Yes but I am pretty sure if you watch the whole video GR you will understand so let's start the video so guy let's start studying GR usage so in very simple words if I tell you GR usage is nothing but an RN Architecture  Ok like there was simple RNN which we read first then there was LSTM which we read last similarly GR use are nothing but RN architecture which we use to process sequential data ok once let us discuss that GR use exists  Why do we do it? So if you remembe

In [40]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents=trans_from_get)

In [41]:
texts

[Document(page_content='Hi Guys my name is Nitesh and you welcome to mine its name is GRU Gated Recurrent Unit so in this video we will understand in great detail first of all if LSTM was performing so well then there is a need to use GR.  Why and after that we will understand its architecture in great detail. To explain the architecture, I will explain with examples and finally we will see the key differences between LSTM and GR usage. Okay, the video may be a little long because it has been explained in great detail.', metadata={'source': 'QQfZAoNGQmE'}),
 Document(page_content="because it has been explained in great detail. Yes but I am pretty sure if you watch the whole video GR you will understand so let's start the video so guy let's start studying GR usage so in very simple words if I tell you GR usage is nothing but an RN Architecture  Ok like there was simple RNN which we read first then there was LSTM which we read last similarly GR use are nothing but RN architecture which w

In [13]:
from langchain.vectorstores import FAISS

In [15]:
class CheckURL(BaseModel):
    url: str
class CheckSearchInput(BaseModel):
    textInput: str

In [16]:
model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


In [42]:
v_store = FAISS.from_documents(documents=texts,embedding=embeddings)

In [43]:
# from langchain_core.prompts import ChatPromptTemplate
# prompt = ChatPromptTemplate.from_template("""
# Answer the following question based only on the provided context. 
# Think step by step before providing a detailed answer. 
# If you don't know the answer, just say that you don't know, don't try to make up an answer.
# <context>
# {context}
# </context>
# Question: {input}""")

In [50]:
prompt_template = """Use the following  text transcript to answer the user's question in detail.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    Context: {context}
    Question: {question}

    Only return the  answer below and nothing else.
    Detailed answer:
    """

In [51]:
prompt = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])

In [32]:
## Create Stuff Docment Chain
# from langchain.chains.combine_documents import create_stuff_documents_chain

In [52]:
model = ChatGoogleGenerativeAI(model="gemini-pro",
                            temperature=0.2,convert_system_message_to_human=True,google_api_key=api)
    

In [53]:

chain_type_kwargs = {"prompt": prompt}
qa = RetrievalQA.from_chain_type(
llm=model,
chain_type="stuff",
retriever=v_store.as_retriever(),
return_source_documents = True,
chain_type_kwargs= chain_type_kwargs,
verbose=True
)


In [54]:
qa("explain story of vikram")["result"]

c:\Users\hp\anaconda3\envs\geminienv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


"Vikram was a powerful king who had an enemy named Kali. They fought, and Vikram was killed. Vikram's son sought revenge but was also killed. Finally, Vikram's grandson grew up and killed Kali, avenging his father and grandfather."